In [1923]:
import pandas as pd
import numpy as np

In [1924]:
train = pd.read_csv("./input/train.csv")
test = pd.read_csv("./input/test.csv")

In [1925]:
print(train.shape)
train.head(1)

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [1926]:
print(test.shape)
test.head(1)

(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q


In [1927]:
print("train null count")
print(train.isnull().sum())
print("----------------")
print("test null count")
print(test.isnull().sum())

train null count
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
----------------
test null count
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [1928]:
def setEmbarked(val):
    result = None
    if val == "Q": result = 2
    elif val == "C": result = 1
    else: result = 0
    return result

In [1929]:
train["Embarked"] = train["Embarked"].apply(setEmbarked)
test["Embarked"] = test["Embarked"].apply(setEmbarked)

In [1930]:
def setSex(val):
    result = None
    if val == "male": result = 1
    else: result = 0
    return result

In [1931]:
train["Sex"] = train["Sex"].apply(setSex)
test["Sex"] = test["Sex"].apply(setSex)

In [1932]:
def setCabin(val):
    result = 0
    if pd.isna(val): result = 0
    else: result = 1
    return result

In [1933]:
train["HasCabin"] = train["Cabin"].apply(setCabin)
test["HasCabin"] = test["Cabin"].apply(setCabin)
train = train.drop(columns=["Cabin"], axis=1)
test = test.drop(columns="Cabin", axis=1)

In [1934]:
test["Fare"] = test["Fare"].fillna(test["Fare"].median())

In [1935]:
train["FamilySize"] = train["SibSp"] + train["Parch"]
test["FamilySize"] = test["SibSp"] + test["Parch"]
train = train.drop(columns=["SibSp", "Parch"], axis=1)
test = test.drop(columns=["SibSp", "Parch"], axis=1)

In [1936]:
train["Title"] = train["Name"].str.extract(', ([A-Z][a-z]+)')
test["Title"] = test["Name"].str.extract(', ([A-Z][a-z]+)')

In [1937]:
train["Title"].value_counts() 

Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Mlle          2
Col           2
Major         2
Ms            1
Mme           1
Jonkheer      1
Lady          1
Don           1
Sir           1
Capt          1
Name: Title, dtype: int64

In [1938]:
test["Title"].value_counts() 

Mr        240
Miss       78
Mrs        72
Master     21
Rev         2
Col         2
Dona        1
Ms          1
Dr          1
Name: Title, dtype: int64

In [1939]:
def setTitle(val):
    result = 0
    if val == "Mr": result = 0
    elif val == "Miss": result = 1
    elif val == "Mrs": result = 2
    else: val = 3
    return result

In [1940]:
train["Title"] = train["Title"].apply(setTitle)
test["Title"] = test["Title"].apply(setTitle)

In [1941]:
train = train.drop(columns=["Ticket", "Name"], axis=1)
test = test.drop(columns=["Ticket", "Name"], axis=1)

In [1942]:
train["FareBand"] = pd.cut(train["Fare"], 4)
train[["FareBand", "Survived"]].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

,FareBand,Survived
0,"(-0.512, 128.082]",0.368113
1,"(128.082, 256.165]",0.724138
2,"(256.165, 384.247]",0.666667
3,"(384.247, 512.329]",1.000000


In [1943]:
def setFare(val):
    result = 0
    if val < 128.082: result = 0
    elif val < 256.165: result = 1
    elif val < 384.247: result = 2
    else: result = 3
    return result

In [1944]:
train["Fare"] = train["Fare"].apply(setFare)
test["Fare"] = test["Fare"].apply(setFare)

In [1945]:
train["Fare"].value_counts()

0    853
1     29
2      6
3      3
Name: Fare, dtype: int64

In [1946]:
test["Fare"].value_counts()

0    389
1     21
2      7
3      1
Name: Fare, dtype: int64

In [1947]:
def setAlone(val):
    if val == 0 : return 1
    else: return 0

In [1948]:
train["IsAlone"] = train["FamilySize"].apply(setAlone)
test["IsAlone"] = test["FamilySize"].apply(setAlone)

In [1949]:
train["FamilyBand"] = pd.cut(train["FamilySize"], 4)
train[["FamilyBand", "Survived"]].groupby(['FamilyBand'], as_index=False).mean().sort_values(by='FamilyBand', ascending=True)

,FamilyBand,Survived
0,"(-0.01, 2.5]",0.388750
1,"(2.5, 5.0]",0.409091
2,"(5.0, 7.5]",0.222222
3,"(7.5, 10.0]",0.000000


In [1950]:
def setFamilyBand(val):
    result = 0
    if val<2.5: result = 0
    elif val<5.0: result = 1
    elif val<7.5: result = 2
    else: result = 3
    return result

In [1951]:
train["FamilySize"] = train["FamilySize"].apply(setFamilyBand)
test["FamilySize"] = test["FamilySize"].apply(setFamilyBand)

In [1952]:
train.corr()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,HasCabin,FamilySize,Title,IsAlone
PassengerId,1.000000,-0.005007,-0.035144,0.042939,0.036847,-0.006345,-0.030467,0.019919,-0.027133,-0.029749,0.057462
Survived,-0.005007,1.000000,-0.338481,-0.543351,-0.077221,0.147466,0.106811,0.316912,-0.077962,0.504442,-0.203367
Pclass,-0.035144,-0.338481,1.000000,0.131900,-0.369226,-0.298580,0.045702,-0.725541,0.129124,-0.142629,0.135207
Sex,0.042939,-0.543351,0.131900,1.000000,0.093254,-0.114771,-0.116569,-0.140391,-0.111924,-0.902498,0.303646
Age,0.036847,-0.077221,-0.369226,0.093254,1.000000,0.036128,0.010171,0.249732,-0.274024,0.023614,0.198270
Fare,-0.006345,0.147466,-0.298580,-0.114771,0.036128,1.000000,0.047596,0.259115,0.108431,0.087740,-0.094555
Embarked,-0.030467,0.106811,0.045702,-0.116569,0.010171,0.047596,1.000000,0.013774,-0.079096,0.053073,0.017807
HasCabin,0.019919,0.316912,-0.725541,-0.140391,0.249732,0.259115,0.013774,1.000000,-0.072565,0.132380,-0.158029
FamilySize,-0.027133,-0.077962,0.129124,-0.111924,-0.274024,0.108431,-0.079096,-0.072565,1.000000,0.093462,-0.383488
Title,-0.029749,0.504442,-0.142629,-0.902498,0.023614,0.087740,0.053073,0.132380,0.093462,1.000000,-0.379074


In [1953]:
train_age_yes = train[train["Age"].notna()]
train_age_no = train[train["Age"].isna()]

In [1954]:
import statsmodels.api as sm
x_table = train_age_yes.drop(columns=["PassengerId", "Survived", "FareBand", "FamilyBand", "Age"], axis=1)
y_table = train_age_yes["Age"]
mod = sm.OLS(y_table,x_table)
fii = mod.fit()

display(fii.summary())
p_values = fii.summary2().tables[1]['P>|t|']
display(p_values)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    Age   R-squared (uncentered):                   0.860
Model:                            OLS   Adj. R-squared (uncentered):              0.859
Method:                 Least Squares   F-statistic:                              543.6
Date:                Tue, 24 Mar 2020   Prob (F-statistic):                   7.70e-296
Time:                        00:23:07   Log-Likelihood:                         -2808.1
No. Observations:                 714   AIC:                                      5632.
Df Residuals:                     706   BIC:                                      5669.
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Pclass        -3.5395      0.649     -5.453      0.000      -4.814      -2.265
Sex           31.0040      1.641     18.891      0.000      27.782      34.226
Fare           0.5983      1.543      0.388      0.698      -2.430       3.627
Embarked       1.2640      0.900      1.404      0.161      -0.503       3.031
HasCabin       5.7491      1.311      4.384      0.000       3.174       8.324
FamilySize    -2.6568      1.045     -2.543      0.011      -4.708      -0.606
Title         20.4676      1.067     19.183      0.000      18.373      22.562
IsAlone       10.1379      1.060      9.562      0.000       8.056      12.219
==============================================================================
Omnibus:                       38.978   Durbin-Watson:                   1.916
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               44.284
Skew:                           0.569   Prob(JB):                     2.42e-10
Kurtosis:                       3.439   Cond. No.                         12.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Pclass        6.849652e-08
Sex           9.887017e-65
Fare          6.982216e-01
Embarked      1.606130e-01
HasCabin      1.342559e-05
FamilySize    1.118902e-02
Title         2.482512e-66
IsAlone       1.880502e-20
Name: P>|t|, dtype: float64

In [1955]:
x = train_age_yes[["Pclass", "Title"]]
y = train_age_yes["Age"]

In [1956]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer
score = make_scorer(mean_squared_error)
k_fold = KFold(n_splits=400, shuffle=True, random_state=42)

In [1957]:
lr = LinearRegression()
lr.fit(x, y)
scoring = "mean_squared_error"
score = cross_val_score(lr, x, y, cv=k_fold, n_jobs=1, scoring=score)
print(np.sqrt(score))
print("LinearRegression: {0: .4f}".format(round(np.mean(np.sqrt(score)))))

[21.50537256 10.00106687 20.94381434 18.13494483  2.706367   14.18719955
 10.44192411  6.28738694 15.25371885 12.66290607 15.18712501 11.1765035
  6.12169311  6.45270358 11.56927653 11.26716534  7.19503552 12.11427684
 12.9293316  24.62455324 11.95802162 15.93759046 10.00327208  3.56395517
  7.26778631  4.97267465  8.09514471 18.97148674 17.18505522 21.60899298
 14.62454174 11.89343161 10.91412503 17.38086831  5.51439655 18.86670931
  7.3734207  14.57248787 10.20493209 19.6827506  10.76341776 12.26697874
  3.17261218 15.47814516  8.62019125  4.99826395 13.7929156   2.81703896
 11.52253711 10.84156054  8.82838047 15.62017597 19.16855204  3.85713459
 15.08276483  3.5441666  15.12135891  6.9175167   8.89837377 13.04819602
 17.27888673  3.58526197  8.47249144 10.2928727   5.82081724  2.08924712
  2.67711815 16.50699209  5.67962613 20.18052835  9.96424155 14.02579945
  3.57510437  7.27920238  2.59093222  8.98205758  5.30793849  6.74110367
  5.59604483 19.31065257  2.21470477  2.38362824  9.

In [1958]:
train_age_yes.head(1)

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,HasCabin,FamilySize,Title,FareBand,IsAlone,FamilyBand
0,1,0,3,1,22.0,0,0,0,0,0,"(-0.512, 128.082]",0,"(-0.01, 2.5]"


In [1959]:
y_predict = lr.predict(train_age_no[["Pclass", "Title"]])
train_age_no["Age"] = y_predict
train = pd.concat([train_age_yes, train_age_no])

/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [1960]:
test_age_yes = test[test["Age"].notna()]
test_age_no = test[test["Age"].isna()]

In [1961]:
x_test = test_age_no[["Pclass", "Title"]]
y_predict = lr.predict(x_test)
test_age_no["Age"] = y_predict
test = pd.concat([test_age_yes, test_age_no])

/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1962]:
train["Age"].astype("int")
test["Age"].astype("int")

0      34
1      47
2      62
3      27
4      22
       ..
408    24
410    24
413    25
416    25
417    25
Name: Age, Length: 418, dtype: int64

In [1963]:
train["AgeBand"] = pd.cut(train["Age"], 4)
train[["AgeBand", "Survived"]].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(0.34, 20.315]",0.458101
1,"(20.315, 40.21]",0.364769
2,"(40.21, 60.105]",0.390625
3,"(60.105, 80.0]",0.227273


In [1964]:
def setAge(val):
    if val < 16.336: result = 0
    elif val < 32.252: result = 1
    elif val < 48.168: result = 2
    elif val <  64.084: result = 3
    else: result = 4
    return result

In [1965]:
train["Age"] = train["Age"].apply(setAge)
test["Age"] = test["Age"].apply(setAge)

In [1966]:
# train["ClassAge"] = train["Age"] * train["Pclass"]
# test["ClassAge"] = test["Age"] * test["Pclass"]

In [1967]:
train.corr()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,HasCabin,FamilySize,Title,IsAlone
PassengerId,1.000000,-0.005007,-0.035144,0.042939,0.022969,-0.006345,-0.030467,0.019919,-0.027133,-0.029749,0.057462
Survived,-0.005007,1.000000,-0.338481,-0.543351,-0.036384,0.147466,0.106811,0.316912,-0.077962,0.504442,-0.203367
Pclass,-0.035144,-0.338481,1.000000,0.131900,-0.419593,-0.298580,0.045702,-0.725541,0.129124,-0.142629,0.135207
Sex,0.042939,-0.543351,0.131900,1.000000,0.074045,-0.114771,-0.116569,-0.140391,-0.111924,-0.902498,0.303646
Age,0.022969,-0.036384,-0.419593,0.074045,1.000000,0.073379,-0.050496,0.289338,-0.227879,0.031718,0.136538
Fare,-0.006345,0.147466,-0.298580,-0.114771,0.073379,1.000000,0.047596,0.259115,0.108431,0.087740,-0.094555
Embarked,-0.030467,0.106811,0.045702,-0.116569,-0.050496,0.047596,1.000000,0.013774,-0.079096,0.053073,0.017807
HasCabin,0.019919,0.316912,-0.725541,-0.140391,0.289338,0.259115,0.013774,1.000000,-0.072565,0.132380,-0.158029
FamilySize,-0.027133,-0.077962,0.129124,-0.111924,-0.227879,0.108431,-0.079096,-0.072565,1.000000,0.093462,-0.383488
Title,-0.029749,0.504442,-0.142629,-0.902498,0.031718,0.087740,0.053073,0.132380,0.093462,1.000000,-0.379074


In [1968]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,HasCabin,FamilySize,Title,FareBand,IsAlone,FamilyBand,AgeBand
0,1,0,3,1,1,0,0,0,0,0,"(-0.512, 128.082]",0,"(-0.01, 2.5]","(20.315, 40.21]"
1,2,1,1,0,2,0,1,1,0,2,"(-0.512, 128.082]",0,"(-0.01, 2.5]","(20.315, 40.21]"
2,3,1,3,0,1,0,0,0,0,1,"(-0.512, 128.082]",1,"(-0.01, 2.5]","(20.315, 40.21]"
3,4,1,1,0,2,0,0,1,0,2,"(-0.512, 128.082]",0,"(-0.01, 2.5]","(20.315, 40.21]"
4,5,0,3,1,2,0,0,0,0,0,"(-0.512, 128.082]",1,"(-0.01, 2.5]","(20.315, 40.21]"


In [1969]:
# x = train.drop(columns=["PassengerId", "Survived", "FareBand", "AgeBand", "FamilyBand", "Age", "FamilySize"], axis=1)
x = train.drop(columns=["PassengerId", "Survived", "FareBand", "AgeBand", "FamilyBand", "HasCabin", "Title"], axis=1)
y = train["Survived"]

In [1970]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import lightgbm as lgb

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=200, shuffle=True, random_state=42)

scoring = 'accuracy'

In [1971]:
# lr = RandomForestClassifier()
# lr.fit(x, y)
# scoring = 'accuracy'
# score = cross_val_score(lr, x, y, cv=k_fold, n_jobs=1, scoring=scoring)
# print(score)
# print("Radnom: {0: .4f}".format(round(np.mean(score)*100, 2)))

In [1972]:
# de = DecisionTreeClassifier()
# de.fit(x, y)
# scoring = 'accuracy'
# score = cross_val_score(de, x, y, cv=k_fold, n_jobs=1, scoring=scoring)
# print(score)
# print("DecisionTree: {0: .4f}".format(round(np.mean(score)*100, 2)))

In [1973]:
# sv = SVC()
# sv.fit(x, y)
# scoring = 'accuracy'
# score = cross_val_score(sv, x, y, cv=k_fold, n_jobs=1, scoring=scoring)
# print(score)
# print("SupportVectorMachine: {0: .4f}".format(round(np.mean(score)*100, 2)))

In [1974]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(x, y)
scoring = 'accuracy'
score = cross_val_score(xgb_model, x, y, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)
print("xgboost: {0: .4f}".format(round(np.mean(score)*100, 2)))

[0.6  0.6  0.8  1.   0.8  0.8  1.   1.   0.8  0.4  1.   1.   1.   0.6
 0.8  1.   0.8  0.8  0.8  0.8  0.8  1.   0.6  1.   0.8  1.   0.6  1.
 1.   1.   1.   0.6  1.   0.6  1.   0.8  1.   0.8  0.8  0.6  0.8  1.
 0.8  0.6  1.   0.6  0.4  0.8  1.   1.   0.4  0.8  1.   0.8  0.4  0.8
 1.   1.   0.4  0.6  0.8  1.   0.8  1.   0.8  0.6  0.6  0.8  0.6  0.4
 1.   0.8  0.8  0.4  0.8  0.8  0.6  0.8  0.8  1.   0.6  1.   0.6  0.8
 1.   0.6  1.   0.8  0.6  0.8  1.   1.   1.   0.75 0.5  0.75 0.75 0.75
 1.   0.5  0.75 1.   0.5  1.   0.75 1.   0.75 0.75 0.75 0.75 1.   0.75
 0.75 0.5  0.75 0.75 1.   1.   1.   0.75 0.5  0.75 0.5  0.75 1.   0.5
 1.   0.75 1.   1.   1.   0.75 1.   0.75 1.   1.   0.75 1.   0.75 1.
 1.   1.   0.75 0.75 1.   0.75 0.75 0.5  0.75 1.   1.   1.   0.75 0.75
 1.   0.75 0.75 1.   1.   1.   0.75 1.   1.   1.   0.75 0.75 0.5  0.75
 0.75 1.   0.5  1.   1.   1.   1.   1.   1.   0.75 1.   0.25 1.   1.
 1.   1.   0.75 1.   0.75 0.75 0.5  1.   0.75 0.5  1.   0.5  0.75 1.
 0.75 1.   0.75 1.  ]

In [1975]:
lgb_model = lgb.LGBMClassifier(objective="binary", random_state=42)
lgb_model.fit(x, y)
scoring = 'accuracy'
score = cross_val_score(lgb_model, x, y, cv=k_fold, n_jobs=1, scoring=scoring)
print(score)
print("lightgbm: {0: .4f}".format(round(np.mean(score)*100, 2)))

[0.6  0.6  0.8  1.   0.8  0.6  1.   1.   0.8  0.4  1.   1.   1.   0.6
 0.8  1.   1.   0.8  0.8  1.   0.8  1.   0.6  1.   0.8  1.   0.6  1.
 0.8  1.   1.   0.6  1.   0.6  1.   0.8  1.   0.8  0.8  0.6  0.8  1.
 0.8  0.6  1.   0.6  0.4  0.8  1.   1.   0.6  0.8  1.   1.   0.4  0.8
 1.   1.   0.4  0.8  0.6  1.   0.8  1.   0.8  0.6  0.6  0.8  0.6  0.4
 1.   0.8  0.8  0.4  0.8  0.8  0.6  0.8  0.8  1.   0.6  1.   0.8  0.8
 1.   0.6  1.   0.8  0.6  0.8  1.   1.   1.   0.75 0.75 0.75 0.75 0.75
 1.   0.5  0.75 1.   0.5  1.   0.75 1.   0.75 0.75 0.75 0.75 1.   0.75
 0.75 0.5  0.75 0.75 1.   1.   1.   0.75 0.5  0.75 0.5  0.75 0.75 0.5
 1.   0.75 1.   1.   1.   0.75 1.   1.   1.   1.   0.75 1.   0.75 1.
 1.   1.   0.75 0.75 1.   0.75 0.75 0.5  0.75 1.   1.   1.   0.75 0.75
 0.75 0.75 0.75 1.   1.   1.   0.75 1.   0.75 1.   0.75 0.75 0.5  0.75
 0.75 1.   0.5  1.   1.   1.   1.   1.   1.   0.75 1.   0.25 1.   1.
 1.   1.   0.75 1.   0.75 0.75 0.5  1.   0.75 0.5  1.   0.5  0.75 1.
 0.75 1.   0.75 1.  ]

In [1976]:
x_test = test.drop(columns=["PassengerId", "HasCabin", "Title"], axis=1)

In [1977]:
predict = lgb_model.predict(x_test)

In [1978]:
submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": predict
})

In [1979]:
submission.to_csv("0324_FINAL_LightGBM_{0: .4f}.csv".format(81.85), index=False)